In [95]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import xgboost as xgb

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

Getting Files open and into dataframes

In [96]:
file_path = "playground-series-s4e11//"

df_train = pd.read_csv(file_path + "train.csv")
df_test = pd.read_csv(file_path + "test.csv" )

Truncating df_train

In [97]:
#DF Train

df_train['Occupation Satisfaction'] = df_train['Study Satisfaction'].combine_first(df_train['Job Satisfaction'])
df_train['Overall Pressure'] = df_train['Academic Pressure'].combine_first(df_train['Work Pressure'])
df_train = df_train.drop(['Study Satisfaction','Job Satisfaction','Academic Pressure', 'Work Pressure','CGPA'], axis=1)


df_train_modified = df_train.drop(['Name', 'Depression','id','City'], axis=1)
df_dummy_train = pd.get_dummies(df_train_modified,drop_first=True,dtype=float)


y = df_train['Depression']

In [98]:
df_dummy_train

,Age,Work/Study Hours,Financial Stress,Occupation Satisfaction,Overall Pressure,Gender_Male,Working Professional or Student_Working Professional,Profession_Accountant,Profession_Analyst,Profession_Architect,...,Degree_S.Pharm,Degree_S.Tech,Degree_UX/UI Designer,Degree_Unite,Degree_Veda,Degree_Vivaan,Degree_Vrinda,Degree_Working Professional,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes
0,49.0,1.0,2.0,2.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26.0,7.0,3.0,3.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,33.0,3.0,1.0,2.0,5.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,22.0,10.0,1.0,1.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,30.0,9.0,4.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140695,18.0,2.0,4.0,4.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
140696,41.0,6.0,5.0,4.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
140697,24.0,4.0,4.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140698,49.0,10.0,1.0,2.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Truncating df_test

In [99]:
df_test['Occupation Satisfaction'] = df_test['Study Satisfaction'].combine_first(df_test['Job Satisfaction'])
df_test['Overall Pressure'] = df_test['Academic Pressure'].combine_first(df_test['Work Pressure'])
df_test = df_test.drop(['Study Satisfaction','Job Satisfaction','Academic Pressure', 'Work Pressure','CGPA'], axis=1)


df_test_modified = df_test.drop(['Name','id','City'], axis=1)
df_dummy_test = pd.get_dummies(df_test_modified,drop_first=True,dtype=float)
df_dummy_test



,Age,Work/Study Hours,Financial Stress,Occupation Satisfaction,Overall Pressure,Gender_Male,Working Professional or Student_Working Professional,Profession_3M,Profession_Accountant,Profession_Analyst,...,Degree_PhD,Degree_Pune,Degree_RCA,Degree_Rupak,Degree_S.Pharm,Degree_Travel Consultant,Degree_Vibha,Degree_Vrinda,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes
0,53.0,9.0,3.0,5.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,58.0,6.0,4.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,53.0,12.0,4.0,1.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,23.0,10.0,4.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,47.0,3.0,4.0,5.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93795,49.0,2.0,2.0,5.0,3.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
93796,29.0,11.0,3.0,1.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
93797,24.0,7.0,1.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93798,23.0,7.0,5.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [100]:
# Get all columns present in either df_train or df_test
def align_dataframes(df1, df2, fill_value=0):
    # Get the union of all columns
    all_columns = set(df1.columns).union(set(df2.columns))
    
    # Add missing columns and fill with `fill_value`
    for col in all_columns:
        if col not in df1.columns:
            df1[col] = fill_value
        if col not in df2.columns:
            df2[col] = fill_value
            
    # Reorder columns to ensure consistency
    df1 = df1[sorted(all_columns)]
    df2 = df2[sorted(all_columns)]
    
    return df1, df2

df_dummy_train, df_dummy_test = align_dataframes(df_dummy_train, df_dummy_test)

df_dummy_train

/var/folders/xs/7cmd8k6907zbj8nkv6m694rc0000gn/T/ipykernel_55332/878029721.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[col] = fill_value
/var/folders/xs/7cmd8k6907zbj8nkv6m694rc0000gn/T/ipykernel_55332/878029721.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[col] = fill_value
/var/folders/xs/7cmd8k6907zbj8nkv6m694rc0000gn/T/ipykernel_55332/878029721.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

,Age,Degree_20,Degree_24,Degree_29,Degree_3.0,Degree_5.56,Degree_5.61,Degree_5.65,Degree_5.88,Degree_7.06,...,Sleep Duration_More than 8 hours,Sleep Duration_No,Sleep Duration_Pune,Sleep Duration_Sleep_Duration,Sleep Duration_Unhealthy,Sleep Duration_Vivan,Sleep Duration_Work_Study_Hours,Sleep Duration_than 5 hours,Work/Study Hours,Working Professional or Student_Working Professional
0,49.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,1.0
1,26.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,7.0,1.0
2,33.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3.0,0.0
3,22.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,10.0,1.0
4,30.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140695,18.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2.0,1.0
140696,41.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,6.0,1.0
140697,24.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,4.0,1.0
140698,49.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,10.0,1.0


In [101]:
x_train, x_test, y_train, y_test = train_test_split(df_dummy_train, y, stratify=y, random_state=23)

In [ ]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( x_train, y_train), ( x_test, y_test)]
    
    clf.fit(x_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(x_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 10,
                        trials = trials)
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [102]:
model = xgb.XGBClassifier(learning_rate =0.1,
    n_estimators=1000,
    early_stopping_rounds=50,
    max_depth=18,
    min_child_weight=9,
    gamma=6.5,
    colsample_bytree=0.6,
    subsample=0.65,
    reg_alpha=166,
    reg_lambda = 0.88,
    objective= 'multi:softprob',
    nthread=-1,
    random_state=42,
    num_class = 2)


In [103]:
model.fit(x_train, y_train, verbose=True, eval_set=[(x_test, y_test)])
y_pred = model.predict(x_test)

[0]	validation_0-mlogloss:0.63694
[1]	validation_0-mlogloss:0.57935
[2]	validation_0-mlogloss:0.53095
[3]	validation_0-mlogloss:0.49388
[4]	validation_0-mlogloss:0.45881
[5]	validation_0-mlogloss:0.42824
[6]	validation_0-mlogloss:0.40689
[7]	validation_0-mlogloss:0.38171
[8]	validation_0-mlogloss:0.36023
[9]	validation_0-mlogloss:0.34403
[10]	validation_0-mlogloss:0.32721
[11]	validation_0-mlogloss:0.31420
[12]	validation_0-mlogloss:0.30137
[13]	validation_0-mlogloss:0.28961
[14]	validation_0-mlogloss:0.27930
[15]	validation_0-mlogloss:0.27018
[16]	validation_0-mlogloss:0.26155
[17]	validation_0-mlogloss:0.25391
[18]	validation_0-mlogloss:0.24687
[19]	validation_0-mlogloss:0.24103
[20]	validation_0-mlogloss:0.23576
[21]	validation_0-mlogloss:0.23062
[22]	validation_0-mlogloss:0.22612
[23]	validation_0-mlogloss:0.22207
[24]	validation_0-mlogloss:0.21850
[25]	validation_0-mlogloss:0.21501
[26]	validation_0-mlogloss:0.21176
[27]	validation_0-mlogloss:0.20869
[28]	validation_0-mlogloss:0.2

In [104]:
y_pred_labels = np.argmax(y_pred, axis=1)
print(f'y_test shape: {y_test.shape}')
print(f'y_pred shape: {y_pred_labels.shape}')

y_test shape: (35175,)
y_pred shape: (35175,)


In [105]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred_labels)
print(f'Accuracy: {accuracy:.2f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred_labels))

Accuracy: 0.94

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     28783
           1       0.85      0.79      0.82      6392

    accuracy                           0.94     35175
   macro avg       0.90      0.88      0.89     35175
weighted avg       0.94      0.94      0.94     35175



In [106]:
importances = model.feature_importances_
columns = df_dummy_train.columns
i = 0

while i< len(columns):
    if importances[i]*100 > 1.0:
        print (f" The importance of feature '{columns[i]}' is {round(importances[i]*100, 2)}%.")
    i += 1

 The importance of feature 'Age' is 25.52%.
 The importance of feature 'Degree_Class 12' is 6.5%.
 The importance of feature 'Dietary Habits_Healthy' is 1.01%.
 The importance of feature 'Dietary Habits_Unhealthy' is 1.78%.
 The importance of feature 'Financial Stress' is 3.46%.
 The importance of feature 'Have you ever had suicidal thoughts ?_Yes' is 18.41%.
 The importance of feature 'Occupation Satisfaction' is 2.0%.
 The importance of feature 'Overall Pressure' is 5.59%.
 The importance of feature 'Profession_Teacher' is 5.47%.
 The importance of feature 'Work/Study Hours' is 2.3%.
 The importance of feature 'Working Professional or Student_Working Professional' is 15.34%.


Predicting x_test now

In [110]:
predictions = model.predict(df_dummy_test)
predictions = np.argmax(predictions, axis=1)

In [111]:
df_submission = pd.DataFrame({
    'id':df_test['id'],
    'Depression': predictions
})

In [112]:
df_submission.to_csv("Submission#1_XGboost.csv", index=False)